# Testing Non-Linear Products Valuation

This notebook walks through:

1. Loading historical index fixings  
2. Building a simple two-index yield curve  
3. Building a dummy SABR volatility surface  
4. Valuing non-linear products:
   - IBOR caplet  
   - Overnight caplet  
   - IBOR cap/floor wrapper  
   - Overnight cap/floor wrapper  
   - IBOR swaption  
   - Overnight swaption  

# 1. Imports & Setup

In [1]:
import sys, os

repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

print("Added to sys.path:", repo_root)

import pandas as pd
from yield_curve import YieldCurve
from sabr import SabrModel
from valuation import IndexManager
from valuation import ValuationEngineRegistry
from product import (
    ProductIborCapFloorlet,
    ProductOvernightCapFloorlet,
    ProductIborCapFloor,
    ProductOvernightCapFloor,
    ProductIborSwaption,
    ProductOvernightSwaption
)
from analytics import SABRCalculator
print("Setup complete.")


Added to sys.path: c:\Users\neels\OneDrive\Desktop\Capstone_Project\FixedIncomeLib
Setup complete.


## 2) Load Index Fixings

On first call, `IndexManager.instance()` will read **`fixing/fixings.csv`** and populate its in-memory registry.

In [2]:
mgr = IndexManager.instance()
print("Loaded indices:", list(mgr._fixings.keys()))


Loaded indices: ['SOFR-1B', 'USD-LIBOR-3M', 'SONIA-1B']


## 3) Build Dummy Yield Curve

We use two indices (SOFR-1B and USD-LIBOR-BBA-1M) with piecewise-constant interpolation.


In [3]:
curve_data = [
    ["SOFR-1B",          "1M", 0.0020],
    ["SOFR-1B",          "3M", 0.0025],
    ["USD-LIBOR-BBA-1M", "1M", 0.0030],
    ["USD-LIBOR-BBA-1M", "3M", 0.0035]
]
df_curve = pd.DataFrame(curve_data, columns=["INDEX","AXIS1","VALUES"])

build_methods_curve = [
    {"TARGET":"SOFR-1B",           "INTERPOLATION METHOD":"PIECEWISE_CONSTANT"},
    {"TARGET":"USD-LIBOR-BBA-1M",  "INTERPOLATION METHOD":"PIECEWISE_CONSTANT"}
]

valueDate = "2025-06-26"

yc = YieldCurve(valueDate, df_curve, build_methods_curve)
print("YieldCurve built. Components:", yc.components.keys())


YieldCurve built. Components: dict_keys(['SOFR-1B', 'USD-LIBOR-BBA-1M'])


## 4) Build Dummy SABR Model

We’ll calibrate a flat SABR surface for USD-LIBOR-BBA-1M so that our caplets can be priced.

In [4]:
sabr_libor = pd.DataFrame([
    ["USD-LIBOR-BBA-1M", 0.25, 0.25, 0.015, 0.5, 0.2, -0.3],
    ["USD-LIBOR-BBA-1M", 0.25, 1.00, 0.017, 0.5, 0.2, -0.3],
    ["USD-LIBOR-BBA-1M", 1.00, 0.25, 0.018, 0.5, 0.2, -0.3],
    ["USD-LIBOR-BBA-1M", 1.00, 1.00, 0.020, 0.5, 0.2, -0.3],
], columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

sabr_sofr = pd.DataFrame([
    ["SOFR-1B", 0.25, 0.25, 0.010, 0.5, 0.2, -0.1],
    ["SOFR-1B", 0.25, 1.00, 0.012, 0.5, 0.2, -0.1],
    ["SOFR-1B", 1.00, 0.25, 0.013, 0.5, 0.2, -0.1],
    ["SOFR-1B", 1.00, 1.00, 0.015, 0.5, 0.2, -0.1],
], columns=sabr_libor.columns)

sabr_data = pd.concat([sabr_libor, sabr_sofr], ignore_index=True)

build_methods_sabr = []
for idx in ("USD-LIBOR-BBA-1M", "SOFR-1B"):
    for param in ("NORMALVOL","BETA","NU","RHO"):
        build_methods_sabr.append({
            "TARGET":        idx,
            "AXIS1":         "AXIS1",
            "AXIS2":         "AXIS2",
            "VALUES":        param,
            "INTERPOLATION": "LINEAR",
            "SHIFT": 0.0,
            "VOL_DECAY_SPEED":  0.2
        })

sabr = SabrModel(
    valueDate,
    sabr_data,
    build_methods_sabr,
    yc
)

print("SabrModel initialized. Components:", sabr.components.keys())


SabrModel initialized. Components: dict_keys(['USD-LIBOR-BBA-1M-NORMALVOL', 'USD-LIBOR-BBA-1M-BETA', 'USD-LIBOR-BBA-1M-NU', 'USD-LIBOR-BBA-1M-RHO', 'SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


## 5) Valuation: IBOR Caplet


In [5]:
caplet_ibor = ProductIborCapFloorlet(
    startDate   ="2025-07-01",
    endDate     ="2025-10-01",
    index       ="USD-LIBOR-BBA-1M",
    optionType  ="CAP",
    strike      =0.02,
    notional    =1000000,
    longOrShort ="LONG"
)

ve = ValuationEngineRegistry().new_valuation_engine(
    sabr,
    {"SABR_METHOD": None},
    caplet_ibor
)
ve.calculateValue()
print("IBOR Caplet PV:", ve.value)


IBOR Caplet PV: ['USD', np.float64(4.688928821898315e-06)]


## 6) Valuation: Overnight Caplet


In [6]:
caplet_ois = ProductOvernightCapFloorlet(
    effectiveDate="2025-07-01",
    termOrEnd    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="CAP",
    strike       =0.018,
    notional     =1000000,
    longOrShort  ="LONG"
)

ve = ValuationEngineRegistry().new_valuation_engine(
    sabr,
    {"SABR_METHOD": "top-down"},
    caplet_ois
)
ve.calculateValue()
print("Overnight Caplet PV:", ve.value)


Overnight Caplet PV: ['USD', np.float64(71.36815634520575)]


## 7) Valuation: IBOR Cap/Floor Wrapper


In [7]:
cap = ProductIborCapFloor(
    effectiveDate="2025-07-01",
    maturityDate ="2026-07-01",
    frequency    ="3M",
    index        ="USD-LIBOR-BBA-1M",
    optionType   ="FLOOR",
    strike       =0.017,
    notional     =2_000_000,
    longOrShort  ="SHORT"
)

ve = ValuationEngineRegistry().new_valuation_engine(
    sabr,
    {"SABR_METHOD": None},
    cap
)
ve.calculateValue()
print("IBOR Cap/Floor PV:", ve.value)


IBOR Cap/Floor PV: ['USD', np.float64(-31447.41769360504)]


## 8) Valuation: Overnight Cap/Floor Wrapper


In [8]:
ois_cap = ProductOvernightCapFloor(
    effectiveDate="2025-07-01",
    maturityDate ="2026-07-01",
    frequency    ="3M",
    index        ="SOFR-1B",
    compounding  ="COMPOUND",
    optionType   ="FLOOR",
    strike       =0.016,
    notional     =1000000,
    longOrShort  ="SHORT"
)

ve = ValuationEngineRegistry().new_valuation_engine(
    sabr,
    {"SABR_METHOD": None},
    ois_cap
)
ve.calculateValue()
print("Overnight Cap/Floor PV:", ve.value)


Overnight Cap/Floor PV: ['USD', np.float64(-14886.439304164971)]


## 9) Valuation: IBOR Swaption


In [9]:
swaption_ibor = ProductIborSwaption(
    optionExpiry="2025-12-01",
    swapStart   ="2026-01-01",
    swapEnd     ="2031-01-01",
    frequency   ="3M",
    iborIndex   ="USD-LIBOR-BBA-1M",
    strikeRate  =0.0175,
    notional    =1000000,
    longOrShort ="LONG",
    optionType  ="PAYER"
)

ve = ValuationEngineRegistry().new_valuation_engine(
    sabr,
    {"SABR_METHOD": None},
    swaption_ibor
)
ve.calculateValue()
print("IBOR Swaption PV:", ve.value)

IBOR Swaption PV: ['USD', np.float64(17064241489.76993)]


## 10) Valuation: Overnight Swaption


In [10]:
swaption_ois = ProductOvernightSwaption(
    optionExpiry  ="2025-10-01",
    swapStart     ="2025-11-01",
    swapEnd       ="2027-11-01",
    frequency     ="1M",
    overnightIndex="SOFR-1B",
    strikeRate    =0.01,
    notional      =500000,
    longOrShort   ="SHORT",
    optionType    ="PAYER"
)

ve = ValuationEngineRegistry().new_valuation_engine(
    sabr,
    {"SABR_METHOD": "top-down"},
    swaption_ois
)
ve.calculateValue()
print("Overnight Swaption PV:", ve.value)


Overnight Swaption PV: ['USD', np.float64(20316412353.452965)]
